# Building Initial Models

My goal for this notebook is to understand how much signal can be extracted from the genes most correlated to the protein presence and if linear models are an appropriate tool for this vector space. Further, the problem statement involves producing different models for 140 continuous targets, namely the proteins whose presence was recorded. This initial model should provide a sense of which proteins may be more challenging to model.

## Imports

In [1]:
import library as lb

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

## Functions

In [130]:
font_title_defaults = {'fontsize': 36,
                   'color': 'black',
                   'alpha': 1,
                   'horizontalalignment': 'center',
                   'rotation': 0,
                   'family': 'serif'}

font_xlabel_defaults = {'fontsize': 24,
                   'color': 'black',
                   'alpha': 1,
                   'horizontalalignment': 'center',
                   'rotation': 0,
                   'family': 'serif'}

font_ylabel_defaults = {'fontsize': 24,
                   'color': 'black',
                   'alpha': 1,
                   'horizontalalignment': 'center',
                   'rotation': 90,
                   'family': 'serif'}


In [134]:
# This handles creating all my fitted models for each of the 140 different proteins.
def fit_and_evaluate_citeseq_models(pipe, selected_genes, X_train, X_test, y_train, y_test, figures = False, pca_viz = True, eval_lr_coefs = False, cross_val = False):
    # Intialize list for the data so that it can be placed in a DataFrame later
    train_r_squared = []
    train_mean_squared_error = []
    train_mean_absolute_error = []
    
    test_r_squared = []
    test_mean_squared_error = []
    test_mean_absolute_error = []
    
    cross_validation_scores = {}
    lr_coefs = {}
    
    loops = 0
    
    # Loops through all 140 proteins.
    # Note this assumes the metadata is still present
    # We all also filter the predictive columns by the selected genes variable.
    for protein, gene_cols in zip(y_train.columns[4:], selected_genes): 
        # Fit the model.
        y_true = y_train[protein]      
        pipe.fit(X_train[gene_cols], y_true)
        y_preds = pipe.predict(X_train[gene_cols])
        
        # Evaluate Model with train data
        train_mean_squared_error.append(mean_squared_error(y_true, y_preds))
        train_r_squared.append(r2_score(y_true, y_preds))
        train_mean_absolute_error.append(mean_absolute_error(y_true, y_preds))
        
        # Apply Cross Validation
        if cross_val:
            cross_validation_scores[protein] = cross_validate(pipe, X_train[selected_genes[0]], y_true, return_train_score = True)
        
        # Overwrite the true values with the test data.
        y_true = y_test[protein]
        y_preds = pipe.predict(X_test[gene_cols])
    
        # Evaluate Model with test data
        test_mean_squared_error.append(mean_squared_error(y_true, y_preds))
        test_r_squared.append(r2_score(y_true, y_preds))
        test_mean_absolute_error.append(mean_absolute_error(y_true, y_preds))
        
        # Create Evaluatation Visualizations the model
        if figures:
            visualizations_linear_regression_model_evaluation(pipe, y_true, y_preds, protein, pca_viz)
        
        # Gather Coefs
        if eval_lr_coefs:
            lr_coefs[protein] = pd.DataFrame({'Coefficients': pipe['lr'].coef_, }, index = selected_genes[0])\
                                    .sort_values(by = 'Coefficients', ascending = False)

        if (loops % 14) == 0:
            print(f'{loops/140*100}% complete')
        loops += 1
    
    pipe_scores = pd.DataFrame({'Train R-Squared' : train_r_squared,
                              'Train Mean-Squared Error' : train_mean_squared_error,
                              'Train Mean-Absolute Error' : train_mean_absolute_error,
                              'Test R-Squared' : test_r_squared,
                              'Test Mean-Squared Error' : test_mean_squared_error,
                              'Test Mean-Absolute Error' : test_mean_absolute_error,
                             }, index = y_train.columns[4:])
    
    return pipe_scores, cross_validation_scores, lr_coefs

# This produces plots as the models are fit.
# The information used to create these plots is lost when running this script. 
# The only way to create the plots is to do so concurrently.
def visualizations_linear_regression_model_evaluation(pipe, y_true, y_preds, name, pca_viz = True):
    residuals = y_true - y_preds
    
    # Residual Plots
    fig, ax = plt.subplots(figsize = (12, 8))

    sns.residplot(x = y_preds,
                  y = residuals,
                  color = 'tab:cyan',
                  lowess = True,
                  line_kws = {'color': 'green'})

    ax.set_title('Residuals Plot', fontdict = font_title_defaults)
    ax.set_xlabel('Predicted Value', fontdict = font_xlabel_defaults)
    ax.set_ylabel('Residual Value', fontdict = font_ylabel_defaults)

    fig.savefig('./figures/residuals/' + 'residual_plot_' + name + '.png')
    plt.close()
    
    # PCA Plots
    if pca_viz:
        pca_var_ratio = pipe['pca'].explained_variance_ratio_

        # explained variance of components
        # This is from the PCA GA lecture
        fig, ax = plt.subplots(figsize=(8,6))
        ax.plot(range(1,51), pca_var_ratio, lw=2)
        ax.scatter(range(1, 51), pca_var_ratio, s=100)
        ax.set_title('Explained Variance of Components', fontdict = font_title_defaults)
        ax.set_xlabel('Principal Component', fontdict = font_xlabel_defaults)
        ax.set_ylabel('Explained Variance', fontdict = font_ylabel_defaults);

        fig.savefig('./figures/pca_explained_variance/' + 'pca_exp_var_plot_' + name + '.png')
        plt.close()

        # cumulative explained variance of components
        # This is from the PCA GA lecture
        cum_var_exp = np.cumsum(pca_var_ratio) * 100
        fig, ax = plt.subplots(figsize=(9,7))
        component_number = range(1, 51)
        ax.plot(component_number, cum_var_exp, lw=7)
        ax.axhline(y=0, linewidth=5, color='grey', ls='dashed')
        ax.axhline(y=100, linewidth=3, color='grey', ls='dashed')
        ax.set_xlim([1,51])
        ax.set_ylim([-5,105])
        ax.set_ylabel('Cumulative Variance Explained', fontdict = font_ylabel_defaults)
        ax.set_xlabel('Component', fontdict = font_xlabel_defaults)  
        ax.set_title('Component vs Cumulative Variance Explained\n', fontdict = font_title_defaults);

        fig.savefig('./figures/pca_cumulative_explained_variance/' + 'pca_cummulative_exp_var_plot_' + name + '.png')
        plt.close()

## Load Data

In [2]:
X_train = pd.read_hdf('./data/train_test_split/X_train_cite_seq.h5')
X_test = pd.read_hdf('./data/train_test_split/X_test_cite_seq.h5')
Y_train = pd.read_hdf('./data/train_test_split/Y_train_cite_seq.h5')
Y_test = pd.read_hdf('./data/train_test_split/Y_test_cite_seq.h5')

In [3]:
Y_train.drop(columns = 'to_stratify', inplace = True)
Y_test.drop(columns = 'to_stratify', inplace = True)
# Created during train-test split, not relevent to modeling

In [4]:
corrs = pd.read_csv('./data/train_test_split/cite_seq_train_protein_gene_corrs.csv')

### Reduce the number of cells considered

I will only consider the latest day, day four, for these models in order to tune them. I will consider all the data in a Google Colab Notebook.

In [5]:
train_mask = Y_train['day'] == 2
test_mask = Y_test['day'] == 2

In [6]:
X_train = X_train[train_mask]
X_test = X_test[test_mask]
Y_train = Y_train[train_mask]
Y_test = Y_test[test_mask]

In [7]:
measure_of_all_data = X_train.shape[0] + X_test.shape[0]
X_train.shape[0] / measure_of_all_data, X_test.shape[0] / measure_of_all_data

(0.8019779418466867, 0.19802205815331328)

The train-test split was stratified on day 4 but still the distribution between train and test is still about 80/20 with respect to day 2.
The Kaggle competition's private testing set contains only data acquired on day 7. My plan had been to fit models on day 4 to be better equiped to handle that variation of the data. But after completing more EDA I transitioned my plan to focus on day two instead.

In [8]:
Y_train['day'].unique()[0], Y_test['day'].unique()[0]

(2, 2)

The mask was applied correctly on the targets, only day 2 is present.

In [9]:
all_true = True
for i in Y_train.index == X_train.index:
    if not i:
        all_true = False
        break
all_true

True

In [10]:
all_true = True
for i in Y_test.index == X_test.index:
    if not i:
        all_true = False
        break
all_true

True

The mask was applied corectly on the predictors, each index pair line up correctly.

## Consider the Correlations of the Genes to the Proteins

The correlations were calculated in a seperate notebook using NVIDIA RAPIDS using the same train-test split global to the project.

### Correlation Analysis

#### Drop missing values

In [11]:
corrs.rename(columns = {'Unnamed: 0': 'gene_id'}, inplace = True)

In [12]:
corrs.set_index('gene_id', inplace = True)

In [13]:
corrs.drop(columns = 'to_stratify', inplace = True)
# Created during train-test split, not relevent to modeling

In [14]:
corrs.head()

,CD86,CD274,CD270,CD155,CD112,CD47,CD48,CD40,CD154,CD52,...,CD94,CD162,CD85j,CD23,CD328,HLA-E,CD82,CD101,CD88,CD224
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000121410_A1BG,-0.000022,-0.002550,-0.004659,-0.000762,0.014661,0.014575,-0.000235,0.000490,-0.006290,0.009381,...,0.005153,-0.001545,-0.002458,-0.000402,0.002696,-0.000680,-0.010392,0.009006,-0.018318,-0.001764
ENSG00000268895_A1BG-AS1,0.001822,-0.011199,-0.000575,0.013445,0.014784,0.011216,0.008230,-0.001860,0.002584,0.011600,...,-0.003138,0.014779,-0.005772,0.001068,0.002792,0.005366,-0.000961,0.005244,-0.003580,-0.004096
ENSG00000175899_A2M,0.064626,0.009540,0.021473,0.013768,0.034001,0.009187,0.049535,0.015647,0.009757,0.044291,...,-0.004049,0.003167,0.010101,0.007980,0.150346,0.015810,-0.007537,0.143869,-0.004689,0.021982
ENSG00000245105_A2M-AS1,0.003193,0.011383,0.021819,0.045378,0.069043,0.017083,-0.013548,0.007841,0.015188,0.039427,...,0.010883,-0.001565,0.006488,0.017381,-0.009324,0.013009,0.007729,0.001289,-0.006739,0.035258
ENSG00000166535_A2ML1,0.003951,0.003135,-0.005503,-0.011076,-0.016184,-0.010638,-0.003587,0.000868,-0.004975,-0.007788,...,-0.001255,-0.008089,-0.002540,-0.002290,0.000386,-0.007121,0.001895,-0.002501,0.006183,-0.005017


In [15]:
corrs.isnull().sum().value_counts()

449    140
dtype: int64

There are 449 different genes that never have recorded presence in the cell's transcriptome. So, for all 140 proteins, when calculating the pearson coefficient between these missing in action genes and each protein a null value is the result. So these null values can simply be dropped, we will not be able to extract signal from them.

In [16]:
corrs.shape[0] - corrs.dropna().shape[0]

449

Dropping these nulls values does indeed only drop the 449 genes that are missing in action.

In [17]:
corrs.dropna(inplace = True)

### Select relevent initial Genes

For every protein I am picking the predictive columns that most correlate to the given protein. These correlations have been caculated in seperate notebook.

In [56]:
number_of_genes_to_select = 1000
selected_genes = []
for protein in Y_train.columns[4:]:
    array = corrs.abs()[protein].sort_values(ascending = False).iloc[0:number_of_genes_to_select].index.values
    selected_genes.append(array)
# The '.values' lets us grab the genes names as an array instead of as part of Pandas' index class.

In [57]:
len(selected_genes[0])

1000

In [58]:
len(selected_genes), sum([len(col_names) for col_names in selected_genes]) / len(selected_genes)

(140, 1000.0)

I have a list of genes for each of the 140 portein targets, and each list has a length set by the `number_of_genes_to_select` variable.

In [59]:
X_train.shape, Y_train.shape

((17597, 22050), (17597, 144))

### Consider the correlations between the different genes, the predictive variables.

Given that this is a regression model we have as an assumption that the predictive columns are uncorrelated.

### Consider Predictor Correlations

In [79]:
# I will use the first protein being predicted as a representative.
gene_corrs = X_train[selected_genes[0]].corr()

gene_corrs.head()

gene_id,ENSG00000112799_LY86,ENSG00000114013_CD86,ENSG00000038427_VCAN,ENSG00000182578_CSF1R,ENSG00000197629_MPEG1,ENSG00000260314_MRC1,ENSG00000163694_RBM47,ENSG00000120708_TGFBI,ENSG00000163221_S100A12,ENSG00000165168_CYBB,...,ENSG00000168004_HRASLS5,ENSG00000117525_F3,ENSG00000163938_GNL3,ENSG00000183527_PSMG1,ENSG00000169429_CXCL8,ENSG00000187210_GCNT1,ENSG00000112697_TMEM30A,ENSG00000135241_PNPLA8,ENSG00000099377_HSD3B7,ENSG00000141367_CLTC
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000112799_LY86,1.000000,0.225454,0.311411,0.422866,0.336338,0.163112,0.329164,NaN,NaN,0.307235,...,NaN,0.020676,-0.021414,-0.027568,0.024882,0.061597,0.017739,0.018289,-0.008644,0.024307
ENSG00000114013_CD86,0.225454,1.000000,0.215843,0.177849,0.159957,0.150939,0.120646,NaN,NaN,0.193007,...,NaN,0.041673,0.000372,-0.014559,0.037489,0.032564,0.009590,0.022623,0.003578,0.020858
ENSG00000038427_VCAN,0.311411,0.215843,1.000000,0.276785,0.224661,0.172077,0.194150,NaN,NaN,0.355031,...,NaN,0.040344,-0.023810,-0.019196,-0.004436,0.036814,0.008284,0.004052,0.004459,0.015530
ENSG00000182578_CSF1R,0.422866,0.177849,0.276785,1.000000,0.392760,0.194852,0.264453,NaN,NaN,0.385559,...,NaN,0.015004,-0.026800,-0.032463,0.067233,0.043190,0.016330,0.029594,0.000325,0.019803
ENSG00000197629_MPEG1,0.336338,0.159957,0.224661,0.392760,1.000000,0.226631,0.210704,NaN,NaN,0.371174,...,NaN,0.000545,-0.006287,-0.029083,0.024414,0.032884,0.021304,0.020993,0.002866,0.020524


In [80]:
# I set the diagonal to be null so as to limit its effect on my analysis.
gene_corrs = gene_corrs.applymap(lambda x: np.nan if x == 1.0 else x)

In [85]:
gene_corrs.abs().mean().describe()

count    952.000000
mean       0.047969
std        0.024540
min        0.007259
25%        0.029926
50%        0.044452
75%        0.059670
max        0.139936
dtype: float64

These predicting columns seem to be not terribly correlated as for each gene on average correlates with its neighbors by about 4%.

In [86]:
gene_corrs.abs().max().describe()

count    952.000000
mean       0.258489
std        0.153701
min        0.039521
25%        0.148642
50%        0.217723
75%        0.327765
max        0.923732
dtype: float64

Still many genes seem to correlate measurably with at least one partner. So using PCA for decorrelation has some relevancy.

## Fit Models

### Dummy Models

In [101]:
dumb = DummyRegressor()
dumb_output = fit_and_evaluate_citeseq_models(dumb, selected_genes, X_train, X_test, Y_train, Y_test, figures = False, pca_viz = False, eval_lr_coefs = False)
dumb_output[0].describe()

0.0% complete
10.0% complete
20.0% complete
30.0% complete
40.0% complete
50.0% complete
60.0% complete
70.0% complete
80.0% complete
90.0% complete


,Train R-Squared,Train Mean-Squared Error,Train Mean-Absolute Error,Test R-Squared,Test Mean-Squared Error,Test Mean-Absolute Error
count,140.0,140.000000,140.000000,1.400000e+02,140.000000,140.000000
mean,0.0,4.565786,1.287292,-4.595413e-04,4.697620,1.301770
std,0.0,7.407119,0.963067,5.329047e-04,7.563962,0.970732
min,0.0,0.498654,0.545291,-2.508999e-03,0.532240,0.556340
25%,0.0,0.836131,0.694243,-6.309384e-04,0.844784,0.701095
50%,0.0,1.109119,0.793540,-2.820876e-04,1.160055,0.801347
75%,0.0,4.941117,1.575946,-7.460175e-05,5.041639,1.610665
max,0.0,39.714775,5.012508,-1.495635e-07,39.611668,5.018114


These are my baseline models for all 140 protein targets. Notice that every model has an R-squared of zero; there is sufficient variation in protein occurence for the mean protein presence across all cells to have no predictive value.

### Simple Linear Regression

In [102]:
lr = Pipeline([
                    ('ss', StandardScaler()),
                    ('lr', LinearRegression())
              ])

lr_output = fit_and_evaluate_citeseq_models(lr, selected_genes, X_train, X_test, Y_train, Y_test, figures = False, pca_viz = False, eval_lr_coefs = False)
lr_output[0].describe()

0.0% complete
10.0% complete
20.0% complete
30.0% complete
40.0% complete
50.0% complete
60.0% complete
70.0% complete
80.0% complete
90.0% complete


,Train R-Squared,Train Mean-Squared Error,Train Mean-Absolute Error,Test R-Squared,Test Mean-Squared Error,Test Mean-Absolute Error
count,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000
mean,0.300508,2.229033,0.980771,0.208950,2.569583,1.051469
std,0.188622,2.965077,0.548501,0.221265,3.382061,0.584750
min,0.046777,0.376419,0.479004,-0.126138,0.451352,0.515178
25%,0.149416,0.669165,0.642289,0.032611,0.775679,0.685691
50%,0.243153,0.904104,0.735815,0.134923,1.067941,0.789496
75%,0.443524,2.215012,1.100455,0.385612,2.651944,1.166368
max,0.763320,18.299826,3.340372,0.697769,20.355043,3.539924


In [103]:
lr_output[0].sort_values(by = 'Test R-Squared', ascending = False).head()

,Train R-Squared,Train Mean-Squared Error,Train Mean-Absolute Error,Test R-Squared,Test Mean-Squared Error,Test Mean-Absolute Error
CD41,0.763320,3.940012,1.310895,0.697769,4.640398,1.406797
CD71,0.712752,4.439002,1.535126,0.691302,4.827795,1.622802
CD88,0.691755,2.098924,1.057348,0.677003,2.254635,1.135838
CD36,0.693565,12.169991,2.020799,0.647768,13.844970,2.147001
CD29,0.662194,3.336928,1.351321,0.641336,3.762771,1.460807


This set of models performs reasonablly well even though there is the possibility that there are some correlation issues.

### Linear Regression with PCA

#### Linear Regression with all of the PCA components

In [108]:
lr_with_pca = Pipeline([
                    ('ss', StandardScaler()),
                    ('pca', PCA(random_state = 2022, whiten = True)),
                    ('lr', LinearRegression())
                ])
lr_with_pca_output = fit_and_evaluate_citeseq_models(lr_with_pca, selected_genes, X_train, X_test, Y_train, Y_test, figures = False, pca_viz = False, eval_lr_coefs = False)
lr_with_pca_output[0].describe()

0.0% complete
10.0% complete
20.0% complete
30.0% complete
40.0% complete
50.0% complete
60.0% complete
70.0% complete
80.0% complete
90.0% complete


,Train R-Squared,Train Mean-Squared Error,Train Mean-Absolute Error,Test R-Squared,Test Mean-Squared Error,Test Mean-Absolute Error
count,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000
mean,0.168208,2.743384,1.066076,0.066973,3.118391,1.137518
std,0.439155,4.638843,0.674519,0.516102,5.100926,0.711260
min,-2.945936,0.376419,0.479004,-4.008304,0.451352,0.515178
25%,0.086347,0.727472,0.673020,-0.049800,0.838868,0.725082
50%,0.179480,1.084463,0.794529,0.085059,1.226445,0.854698
75%,0.415975,2.402690,1.181657,0.360225,2.934454,1.253022
max,0.762587,42.683697,5.143827,0.697061,45.776676,5.383176


In [110]:
lr_with_pca_output[0].sort_values(by = 'Test R-Squared', ascending = False).head()

,Train R-Squared,Train Mean-Squared Error,Train Mean-Absolute Error,Test R-Squared,Test Mean-Squared Error,Test Mean-Absolute Error
CD41,0.762587,3.952211,1.314615,0.697061,4.651274,1.407032
CD71,0.712752,4.439002,1.535126,0.691302,4.827794,1.622802
CD88,0.691755,2.098924,1.057348,0.677003,2.254635,1.135838
CD36,0.693049,12.190478,2.027725,0.647109,13.870872,2.155344
CD29,0.662194,3.336928,1.351321,0.641336,3.762770,1.460807


This is fit with PCA but with all of the components. These fits appear to be a bit weaker. The bottom quartile consists of all negative R-squares; whereas, for simple linear regression this is not the case. This adds complexity with limited payout, so I will not move forward with these models.

#### Linear Regression with PCA used for additional dimension reduction

In [109]:
lr_with_pca_dim_reduce = Pipeline([
                    ('ss', StandardScaler()),
                    ('pca', PCA(n_components = 50, random_state = 2022, whiten = True)),
                    ('lr', LinearRegression())
                ])

lr_with_pca_dim_reduce_output = fit_and_evaluate_citeseq_models(lr_with_pca_dim_reduce, selected_genes, X_train, X_test, Y_train, Y_test, figures = False, pca_viz = False, eval_lr_coefs = False)
lr_with_pca_dim_reduce_output[0].describe()

0.0% complete
10.0% complete
20.0% complete
30.0% complete
40.0% complete
50.0% complete
60.0% complete
70.0% complete
80.0% complete
90.0% complete


,Train R-Squared,Train Mean-Squared Error,Train Mean-Absolute Error,Test R-Squared,Test Mean-Squared Error,Test Mean-Absolute Error
count,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000
mean,0.245931,2.473189,1.020075,0.240983,2.537840,1.033717
std,0.191507,3.360340,0.581746,0.198841,3.403917,0.586976
min,0.017739,0.407862,0.495594,-0.013244,0.436904,0.505422
25%,0.095057,0.713116,0.658818,0.080756,0.731315,0.668393
50%,0.181625,0.965422,0.756889,0.175011,1.010699,0.765316
75%,0.387586,2.483458,1.131813,0.384839,2.703307,1.135018
max,0.726816,20.141062,3.489825,0.704332,20.025169,3.515181


In [111]:
lr_with_pca_dim_reduce_output[0].sort_values(by = 'Test R-Squared', ascending = False).head()

,Train R-Squared,Train Mean-Squared Error,Train Mean-Absolute Error,Test R-Squared,Test Mean-Squared Error,Test Mean-Absolute Error
CD41,0.726816,4.547701,1.364343,0.704332,4.539642,1.354156
CD71,0.675666,5.012107,1.634502,0.687391,4.888964,1.633802
CD88,0.650540,2.379570,1.126514,0.679173,2.239488,1.129326
CD62L,0.631762,8.309288,2.233011,0.636367,7.953695,2.186314
CD29,0.621577,3.738151,1.434511,0.634411,3.835426,1.467508


Remarkably even though the number of components are reduced from one thousands to fifty these fits are about as robust as the fits from the simple linear regression model. I will further compare these.

In [124]:
# This computes the difference between the simple lr and lr with pca for dim reduction on the train and test r-squared values.
pd.DataFrame(lr_with_pca_dim_reduce_output[0]['Train R-Squared'] - lr_output[0]['Train R-Squared'])\
    .merge(pd.DataFrame(lr_with_pca_dim_reduce_output[0]['Test R-Squared'] - lr_output[0]['Test R-Squared']), right_index = True, left_index = True).describe()

,Train R-Squared,Test R-Squared
count,140.000000,140.000000
mean,-0.054577,0.032032
std,0.010318,0.026617
min,-0.093303,-0.040188
25%,-0.059073,0.008695
50%,-0.053886,0.034453
75%,-0.049480,0.049394
max,-0.011690,0.112894


In this DataFrame Train R-Squared is the difference between the simple linear regression's train r-squares and the linear regression with pca for dimension reduction's train r-squares and presents the distribution of those values. It does the same for Test R-Squared.

There is remarkable similarity between these two sets of models. That Train R-Squared is negative for all values means that the lr outperforms lr_with_pca_dim_reduce each time. But that Test R-Squared is positive for all values means that the lr_with_pca_dim_reduce outperforms lr each time.

This implies that the lr_with_pca_dim_reduce are less overfit than the lr models.

In [128]:
diff = pd.DataFrame(lr_with_pca_dim_reduce_output[0]['Train R-Squared'] - lr_with_pca_dim_reduce_output[0]['Test R-Squared'])\
    .merge(pd.DataFrame(lr_output[0]['Train R-Squared'] - lr_output[0]['Test R-Squared']), right_index = True, left_index = True).describe()
diff.columns = ['lr_with_pca_dim_reduce_output diff', 'lr_diff']
diff

,lr_with_pca_dim_reduce_output diff,lr_diff
count,140.000000,140.000000
mean,0.004948,0.091557
std,0.029017,0.050047
min,-0.080044,-0.009941
25%,-0.006905,0.058602
50%,0.001208,0.089266
75%,0.011874,0.111359
max,0.148594,0.304592


This is a dataframe that shows the difference between train and test for each of the two sets of models in question.

This shows lr_with_pca_dim_reduce is much less overfit across the board. For this reason I will continue my analysis with only this set of models. It outperforms the others with respect to not being overfit and is much easier to compute as it has only 50 predictive features; whereas, the other models have a thousand.

## Evaluate Models

### Check Residuals and Evaluate PCA

#### Linear Regression Models with PCA for dimension reduction

In [135]:
lr_with_pca_dim_reduce = Pipeline([
                    ('ss', StandardScaler()),
                    ('pca', PCA(n_components = 50, random_state = 2022, whiten = True)),
                    ('lr', LinearRegression())
                ])

lr_with_pca_dim_reduce_output = fit_and_evaluate_citeseq_models(lr_with_pca_dim_reduce, selected_genes, X_train, X_test, Y_train, Y_test, figures = True, pca_viz = True, eval_lr_coefs = False)
lr_with_pca_dim_reduce_output[0].describe()

0.0% complete
10.0% complete
20.0% complete
30.0% complete
40.0% complete
50.0% complete
60.0% complete
70.0% complete
80.0% complete
90.0% complete


,Train R-Squared,Train Mean-Squared Error,Train Mean-Absolute Error,Test R-Squared,Test Mean-Squared Error,Test Mean-Absolute Error
count,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000
mean,0.245931,2.473189,1.020075,0.240983,2.537840,1.033717
std,0.191507,3.360340,0.581746,0.198841,3.403917,0.586976
min,0.017739,0.407862,0.495594,-0.013244,0.436904,0.505422
25%,0.095057,0.713116,0.658818,0.080756,0.731315,0.668393
50%,0.181625,0.965422,0.756889,0.175011,1.010699,0.765316
75%,0.387586,2.483458,1.131813,0.384839,2.703307,1.135018
max,0.726816,20.141062,3.489825,0.704332,20.025169,3.515181


Although this output does not look like much more than what has been shown earlier it does produce additional material. The figures parameter was set to true. In my figures directory you can find three subdirectories:
1. residuals: This now has a residual plot for each of the 140 models. There is a wide variety of types included there and I look forward to more analysis related to these graphs in the future.
1. pca_explained_variance: This now has a plot of the pca_explained_variance_ratio, as done in the PCA lecture, for each of the 140 models.
1. pca_cumulative_explained_variance: This now has a plot of the cumulative pca_explained_variance_ratio, as done in the PCA lecture, for each of the 140 models. There is very little variation here.

The residuals graphs have a wide array of patterns. Clearly several of these models violate the assumptions of regression models on that score in an interesting way. I intend to continue to next expand my analysis to considering those patterns.

When considering the cummulative explained variance the graphs are essentially the same and reach only about 20%. Much of the signal is being lost but apperently it is not needed to produce effective fits.

In [152]:
lr_with_pca_dim_reduce_output[0].sort_values(by = 'Test R-Squared', ascending = False).head(7)

,Train R-Squared,Train Mean-Squared Error,Train Mean-Absolute Error,Test R-Squared,Test Mean-Squared Error,Test Mean-Absolute Error
CD41,0.726816,4.547701,1.364343,0.704332,4.539642,1.354156
CD71,0.675666,5.012107,1.634502,0.687391,4.888964,1.633802
CD88,0.650540,2.379570,1.126514,0.679173,2.239488,1.129326
CD62L,0.631762,8.309288,2.233011,0.636367,7.953695,2.186314
CD29,0.621577,3.738151,1.434511,0.634411,3.835426,1.467508
CD49b,0.635266,7.407298,2.093840,0.631387,7.378318,2.110447
CD45RA,0.612728,2.627689,1.228023,0.618073,2.703185,1.246980


In [162]:
top_performers = lr_with_pca_dim_reduce_output[0].sort_values(by = 'Test R-Squared', ascending = False).head(14).index

## Conclusions

In this notebook, I have evaluated an array of regression models. I picked simple linear regression models because I am trying to fit 140 different targets. Since simple linear regression models have a closed form, this will produce faster fits more reliably and reduce the fine-tuning required for this initial stage. PCA could be a substantive additive that insulated my models from overfitting while reducing the dimensions needed to fit my model. I believe I have made substantive progress in finding the best general linear regression strategy for fitting this many targets. In the future, I would like to analyze the properties of the proteins that are the best fit by this strategy and those poorly fit.

This notebook shows a significant signal in these regression models and that PCA preserves it accurately, even when the number of components retained is small, and the cumulative explained variance remains at around 20%.

In future work, I would like to analyze the residual plots and transition to another modeling strategy that includes what I have learned here (and will continue to learn). My main goal is to build a generalized model that can adapt to the specific needs of the individual targets to get them the essential signals they need.